In [3]:
import cma
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# help(cma.fmin)


In [1]:
def generate_simulated(x1,N,M,params,T0,TN):
        """
        Generate simulated data for Pareto Stochastic Process
        """
        T=np.zeros((N,1)) 
        h=(TN-T0)/N     
        mu=params[0]
        sigma_0=params[1] 
        lam_inv = 1/params[2]
        alpha = params[3]
        sigm = params[4]
        for i in range(0,N-1):
            T[i]=T0+(i)*h
        T[N-1] = TN
        X=np.zeros((N,M)) 
        Y1 = ((T-mu)/sigma_0)**(lam_inv);
        Y2 = 1./np.array(T-mu);
        for j in range(M):
            X[0,j]=x1
            Z=np.random.normal(size=(N-1,1))
            for i in range(N-1):
                 
                X[i+1,j]=X[i,j]- (Y2[i]*lam_inv*(params[2] + alpha - ((1+alpha)/(1+Y1[i])))*X[i,j]*h) + (sigm*(X[i,j])*np.sqrt(h)*Z[i])

                #X[i+1,j]=X[i,j]-((alpha)*X[i,j]*h/(T[i]+beta))+(sigm*(X[i,j])*np.sqrt(h)*Z[i])
        return X, T
def alpha_estimator2(alpha,T,X):
    """
        Estimate Alpha parameter for  pareto stochastic process this method require X to be defined 
    """
    C=np.zeros(X.shape)
    n = len(X)
    Y1 = ((T-alpha[0])/alpha[1])**(1/alpha[2]);
    Y2 = np.array(T-alpha[0]);
    for i in range(1,n):
        C[i]=np.log(X[i]/X[i-1])+ ((alpha[3] +1 )*np.log((1+Y1[i])/(1+Y1[i-1]))) + ((1-1/alpha[2])* np.log(Y2[i]/Y2[i-1]))
    out1=0
    for i in range(1,n):
        out1=out1+((C[i])**2);
    store=out1;
    out1=(out1/(n-1))+1;
    h = T[3]-T[2] 
    sigma_2 = (np.sqrt(out1)-1)*2/h;
#     out=0;
#     out2=0;
#     for i in range(1,n):
#         out2 = out2 + ((C[i]-1+np.sqrt(out1))*(1/((T[i]+alpha[1])*(T[i-1]+alpha[1]))))
#         out=out+((C[i]-1+np.sqrt(out1))*(np.log((T[i]+alpha[1])/(T[i-1]+alpha[1]))))
# #     out3=np.sqrt(out2**2 + out**2)
#     out3 = abs(out2)+ abs(out)
    out2 = ((n-1)/2)*(np.log(sigma_2) +  np.log(2*np.pi*h)) 
    for i in range(1,n):
        out2=out2+ np.log(X[i]) + (1/(2*sigma_2*h))*((C[i] + np.sqrt(out1) -1 )**2)
    return out2
def sigma_estimator2(alpha,T,X):
    """
        Estimate Alpha parameter for  pareto stochastic process this method require X to be defined 
    """
    C=np.zeros(X.shape)
    n = len(X)
    Y1 = ((T-alpha[0])/alpha[1])**(1/alpha[2]);
    Y2 = np.array(T-alpha[0]);
    for i in range(1,n):
        C[i]=np.log(X[i]/X[i-1])- ((alpha[3] +1 )*np.log((1+Y1[i])/(1+Y1[i-1]))) + ((1-1/alpha[2])* np.log(Y2[i]/Y2[i-1]))
    out1=0
    for i in range(1,n):
        out1=out1+((C[i])**2);
    store=out1;
    out1=(out1/(n-1))+1;
    h = T[3]-T[2] 
    sigma_2 = (np.sqrt(out1)-1)*2/h
    return sigma_2

In [4]:
params = [234.89998196400018, 10.777076435849843, 0.07201993061930664, 0.31466442437678666, 0]
T0 = 250
TN = 300
N = 10000
M=1
x1 = 100
Y = generate_simulated(x1,N,M,params,T0,TN)
# plt.plot(Y[1], Y[0])
T  = Y[1]
X = Y[0] 
x0 = [100, 0.1 ,5 ,2] 
sigma0 = 1
options = {'bounds':([None,0, 0 , 0],[T0,10000, 10000, 10000]),'tolfunhist':1e-100,'tolfun': 1e-100, 'maxfevals': 100000,'seed':1234}

In [5]:
res2 = cma.fmin(alpha_estimator2, x0, sigma0,options, args = (T,X))
# sigma_estimator2(params,T,X)

(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=1234, Fri Jan 15 16:14:38 2021)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      8 -5.920924796936092e+04 1.0e+00 8.58e-01  8e-01  9e-01 0:02.4


C:\Users\asus\Anaconda3\lib\site-packages\cma\utilities\utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [-59147.00857229] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    2     16 -5.926439024132762e+04 1.3e+00 8.46e-01  7e-01  8e-01 0:04.8


C:\Users\asus\Anaconda3\lib\site-packages\cma\utilities\utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [-59183.93240701] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    3     24 -5.930886561603769e+04 1.2e+00 8.82e-01  7e-01  9e-01 0:07.6
    5     40 -5.942717363371869e+04 1.6e+00 9.21e-01  7e-01  1e+00 0:12.8
    7     56 -5.983991800451879e+04 2.5e+00 1.48e+00  1e+00  2e+00 0:17.7
    9     72 -6.816894199049198e+04 3.5e+00 2.98e+00  3e+00  5e+00 0:23.1
   12     96 -6.361314306477747e+04 3.8e+00 3.33e+00  3e+00  5e+00 0:30.7


C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: RuntimeWarning: overflow encountered in power
C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in true_divide


   15    120 -6.946266463539802e+04 4.1e+00 1.94e+00  1e+00  3e+00 0:38.2
   19    152 -6.971840207916523e+04 4.1e+00 1.36e+00  8e-01  2e+00 0:48.4
   23    184 -6.976409107655844e+04 5.7e+00 1.26e+00  7e-01  2e+00 0:58.2
   28    224 -6.981794389728102e+04 4.7e+00 1.27e+00  7e-01  1e+00 1:10.2
   33    264 -6.977991610779930e+04 7.1e+00 1.99e+00  1e+00  2e+00 1:22.8
   38    304 -6.817127722254203e+04 8.5e+00 1.09e+00  5e-01  1e+00 1:35.7
   44    352 -6.992300799564763e+04 6.3e+00 8.11e-01  2e-01  5e-01 1:50.6
   51    408 -6.973716108629532e+04 1.0e+01 1.53e+00  2e-01  1e+00 2:06.6
   58    464 -6.993490576270717e+04 1.4e+01 1.32e+00  1e-01  1e+00 2:23.2
   66    528 -6.996844324874817e+04 7.3e+01 1.26e+00  5e-02  2e+00 2:41.0
   74    592 -7.011684855387177e+04 1.9e+02 2.25e+00  1e-01  4e+00 2:58.8
   82    656 -7.051174259052015e+04 5.0e+02 6.60e+00  4e-01  1e+01 3:16.9
   91    728 -7.186337584851899e+04 2.4e+02 1.02e+01  3e-01  1e+01 3:36.8
  100    800 -7.332465424449460e+04 1.

In [7]:
# Y_est = generate_simulated(x1,N,M,np.append(res2[0],0),T0,TN)
# plt.plot(Y[1], Y[0])
# plt.plot(Y_est[1], Y_est[0])
# alpha_estimator2(params,T,X)
# alpha_estimator2(params,T,X)
from scipy import linalg, optimize
res = optimize.minimize(alpha_estimator2, x0, args=(T,X), method='Nelder-Mead')

C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in power
C:\Users\asus\Anaconda3\lib\site-packages\scipy\optimize\optimize.py:605: RuntimeWarning: invalid value encountered in less
  if fxr < fsim[0]:
C:\Users\asus\Anaconda3\lib\site-packages\scipy\optimize\optimize.py:616: RuntimeWarning: invalid value encountered in less
  if fxr < fsim[-2]:
C:\Users\asus\Anaconda3\lib\site-packages\scipy\optimize\optimize.py:621: RuntimeWarning: invalid value encountered in less
  if fxr < fsim[-1]:
C:\Users\asus\Anaconda3\lib\site-packages\scipy\optimize\optimize.py:609: RuntimeWarning: invalid value encountered in less
  if fxe < fxr:


In [73]:
alpha_estimator2(res2[0],T,X)

array([1.94266612e-05])

In [12]:
params

[234.89998196400018,
 10.777076435849843,
 0.07201993061930664,
 0.31466442437678666,
 0]

In [14]:
res.x.tolist()

[234.64906155248343,
 0.0001842405690833945,
 0.84827942989004,
 3.742936816649422]